In [1]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import rnn

In [2]:
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = d2l.load_data_jay_lyrics()

In [3]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()

In [6]:
#init params
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    def _three():
        return (_one((num_inputs, num_hiddens)),
               _one((num_hiddens, num_hiddens)),
               nd.zeros(num_hiddens, ctx=ctx))
    W_xz, W_hz, b_z = _three() #update gate params
    W_xr, W_hr, b_r = _three() #reset gate params
    W_xh, W_hh, b_h = _three() #hidden layer params
    
    #output layer params
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, 
              W_hq, b_q]
    
    for param in params:
        param.attach_grad()
    return params

In [20]:
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [21]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, =state
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + nd.dot(R * H, W_hh) + b_h)
        
        H = Z * H + (1 -Z) * H_tilda
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [22]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [23]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 102.827144, time 0.41 sec
 - 分开 我想要你你的你 我不要再你 我不要再你 我不要再你 我不要再你 我不要再你 我不要再你 我不要再你
 - 不分开 我想要你你的你 我不要再你 我不要再你 我不要再你 我不要再你 我不要再你 我不要再你 我不要再你
epoch 100, perplexity 15.352225, time 0.40 sec
 - 分开不能  没有你在我有多难熬多恼恼  没有你爱你 一场悲剧 你在那里 在小村外的溪边 默默等待 娘子 
 - 不分开 不知不觉 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不
epoch 150, perplexity 2.993008, time 0.40 sec
 - 分开不能米  没有回忆 我试著好抽离 如果伦悄默离离一直 融牵在宇宙里 我每天常小小的课火  杵真文 开
 - 不分开 我知一好你护你说一直 说吃 你爱再这样我想要你 说不去 三沉默 娘子她依在江南等我 泪不休 语沉默
epoch 200, perplexity 1.379487, time 0.40 sec
 - 分开始那雨 思地你手 其人的甜步 还它它停留的白墙酱瓦的淡  古着我跟世是你一透 融化壁酒熟到 一使就著
 - 不分开期 我叫你爸 你打我笑 这样对吗干嘛这样 何必让酒牵鼻子走 瞎 说过了很多 我的认真真给黑色 默默莹
epoch 250, perplexity 1.168218, time 0.40 sec
 - 分开的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
 - 不分开吗 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子走 瞎 说过了没去 我妈得很常二黑色在还前卷
